In [1]:
from langchain_openai import ChatOpenAI
import os
from openai import api_key, base_url
from transformers import Qwen2_5_VLConfig
# llm = ChatOpenAI(model=)
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "sk-C3gsaWJaOsmjrNv1fIobc4zEEsEkD15T98j80Vc8mNuSE420")
# llm = ChatOpenAI(model="o3-mini", api_key=OPENAI_API_KEY, base_url="https://api.deepbricks.ai/v1/")
# QWEN_API_KEY = os.getenv("QWEN_API_KEY", "sk-ypjnechwutigvbyglbkguukzsmzkkxfibauydwkbjrypwojd")
llm = ChatOpenAI(model="Qwen/Qwen3-32B", api_key="sk-qrqzhcsqektnkbfnrszxhfkcvhbpcmqoihoffxsizuvvkiwn", base_url="https://api.siliconflow.cn/v1")

/home/zpc/anaconda3/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

from langchain.prompts import example_selector
examples = [
    {"input": "开心", "output": "伤心"},
    {"input": "快乐", "output": "难过"},
    {"input": "高", "output": "矮"},
    {"input": "精力充沛", "output":"没精打采"},
    {"input": "粗", "output": "细"}
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="输入: {input}\n输出: {output}"
)

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=40
)

dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入的反义词",
    suffix="输入: {adjective}\n输出:",
    input_variables=["adjective"],
    example_separator="\n"
)


In [3]:



new_example = {"input": "胖", "output": "瘦"}

dynamic_prompt.example_selector.add_example(new_example)

dynamic_prompt


FewShotPromptTemplate(input_variables=['adjective'], input_types={}, partial_variables={}, example_selector=LengthBasedExampleSelector(examples=[{'input': '开心', 'output': '伤心'}, {'input': '快乐', 'output': '难过'}, {'input': '高', 'output': '矮'}, {'input': '精力充沛', 'output': '没精打采'}, {'input': '粗', 'output': '细'}, {'input': '胖', 'output': '瘦'}], example_prompt=PromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, template='输入: {input}\n输出: {output}'), get_text_length=<function _get_length_based at 0x7f866d00bb50>, max_length=40, example_text_lengths=[4, 4, 4, 4, 4, 4]), example_prompt=PromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, template='输入: {input}\n输出: {output}'), suffix='输入: {adjective}\n输出:', example_separator='\n', prefix='给出每个输入的反义词')

In [4]:
from pyexpat import model
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

output_parser = StrOutputParser()
chain = dynamic_prompt | llm | output_parser

chain.invoke({"adjective": "幸福"})


'\n\n输出: 不幸'

In [8]:
#相
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector, SemanticSimilarityExampleSelector
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embedding_path = "/home/zpc/langchain/blibli/bge-large-zh-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=embedding_path)
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

examples = [
    {"input": "开心", "output": "伤心"},
    {"input": "快乐", "output": "难过"},
    {"input": "高", "output": "矮"},
    {"input": "精力充沛", "output":"没精打采"},
    {"input": "粗", "output": "细"}
]








/tmp/ipykernel_123070/118519743.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_path)


In [13]:
from langchain_community.vectorstores import FAISS
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples,
    embeddings,
    FAISS,
    k=2
)

mmr_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = "给出每个输入的反义词",
    suffix = "Input: {adjective}\nOutput:",
    input_variables = ["adjective"],
)

In [14]:
print(mmr_prompt.format(adjective="幸福"))

给出每个输入的反义词

Input: 快乐
Output: 难过

Input: 粗
Output: 细

Input: 幸福
Output:


In [16]:
chain = mmr_prompt | llm | output_parser
chain.invoke({"adjective": "幸福"})


'\n\nOutput: 痛苦'

In [19]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
    {
        "question": "乾隆和曹操谁活得更久?",
        "answer": """
这里是否需要跟进问题：是的。
追问：乾隆去世时几岁？
中间答案：乾隆去世时87岁。
追问：曹操去世时几岁？
中间答案：曹操去世时66岁。
所以最终答案是：乾隆
""",
    },
    {
        "question": "小米手机的创始人什么时候出生?",
        "answer": """
这里是否需要跟进问题：是的。
追问：小米手机的创始人是谁？
中间答案：小米手机 由 雷军 创立。
跟进：雷军什么时候出生？
中间答案：雷军出生于 1969 年 12 月 16 日。
所以最终的答案是：1969 年 12 月 16 日
""",
    },
    {
        "question": "乔治·华盛顿的外祖父是谁？",
        "answer": """
这里是否需要跟进问题：是的。
追问：乔治·华盛顿的母亲是谁？
中间答案：乔治·华盛顿的母亲是玛丽·鲍尔·华盛顿。
追问：玛丽·鲍尔·华盛顿的父亲是谁？
中间答案：玛丽·鲍尔·华盛顿的父亲是约瑟夫·鲍尔。
所以最终答案是：约瑟夫·鲍尔
""",
    },
    {
        "question": "《大白鲨》和《皇家赌场》的导演是同一个国家的吗？",
        "answer": """
这里是否需要跟进问题：是的。
追问：《大白鲨》的导演是谁？
中间答案：《大白鲨》的导演是史蒂文·斯皮尔伯格。
追问：史蒂文·斯皮尔伯格来自哪里？
中间答案：美国。
追问：皇家赌场的导演是谁？
中间答案：《皇家赌场》的导演是马丁·坎贝尔。
跟进：马丁·坎贝尔来自哪里？
中间答案：新西兰。
所以最终的答案是：不会
""",
    },
]


example_prompt = PromptTemplate(
    input_variables=["question", "answer"], template="Question: {question}\n{answer}"
)

print(example_prompt.format(**examples[0]))




Question: 乾隆和曹操谁活得更久?

这里是否需要跟进问题：是的。
追问：乾隆去世时几岁？
中间答案：乾隆去世时87岁。
追问：曹操去世时几岁？
中间答案：曹操去世时66岁。
所以最终答案是：乾隆



In [20]:
prompt = FewShotPromptTemplate(
    examples= examples,
    example_prompt=example_prompt,
    suffix="Question: {input}\nAnswer:",
    input_variables=["input"],
)
print(prompt.format(input="李白和白居易谁活得更久"))

Question: 乾隆和曹操谁活得更久?

这里是否需要跟进问题：是的。
追问：乾隆去世时几岁？
中间答案：乾隆去世时87岁。
追问：曹操去世时几岁？
中间答案：曹操去世时66岁。
所以最终答案是：乾隆


Question: 小米手机的创始人什么时候出生?

这里是否需要跟进问题：是的。
追问：小米手机的创始人是谁？
中间答案：小米手机 由 雷军 创立。
跟进：雷军什么时候出生？
中间答案：雷军出生于 1969 年 12 月 16 日。
所以最终的答案是：1969 年 12 月 16 日


Question: 乔治·华盛顿的外祖父是谁？

这里是否需要跟进问题：是的。
追问：乔治·华盛顿的母亲是谁？
中间答案：乔治·华盛顿的母亲是玛丽·鲍尔·华盛顿。
追问：玛丽·鲍尔·华盛顿的父亲是谁？
中间答案：玛丽·鲍尔·华盛顿的父亲是约瑟夫·鲍尔。
所以最终答案是：约瑟夫·鲍尔


Question: 《大白鲨》和《皇家赌场》的导演是同一个国家的吗？

这里是否需要跟进问题：是的。
追问：《大白鲨》的导演是谁？
中间答案：《大白鲨》的导演是史蒂文·斯皮尔伯格。
追问：史蒂文·斯皮尔伯格来自哪里？
中间答案：美国。
追问：皇家赌场的导演是谁？
中间答案：《皇家赌场》的导演是马丁·坎贝尔。
跟进：马丁·坎贝尔来自哪里？
中间答案：新西兰。
所以最终的答案是：不会


Question: 李白和白居易谁活得更久
Answer:


In [21]:
chain = prompt | llm | output_parser
chain.invoke({"input": "李白和白居易谁活得更久？"})

'\n\n这里是否需要跟进问题：是的。  \n追问：李白去世时几岁？  \n中间答案：李白去世时61岁。  \n追问：白居易去世时几岁？  \n中间答案：白居易去世时74岁。  \n所以最终答案是：白居易'

In [28]:
import select
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
examples_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    Chroma,
    k=1
    )

question = "李白和白居易谁活得更久"

selected_examples = examples_selector.select_examples({"question": question})
print(f"最相近的案例是{selected_examples}")

最相近的案例是[{'question': '乾隆和曹操谁活得更久?', 'answer': '\n这里是否需要跟进问题：是的。\n追问：乾隆去世时几岁？\n中间答案：乾隆去世时87岁。\n追问：曹操去世时几岁？\n中间答案：曹操去世时66岁。\n所以最终答案是：乾隆\n'}]


In [31]:
prompt = FewShotPromptTemplate(
    example_selector=examples_selector,
    example_prompt=example_prompt,
    prefix = "请根据案例的方式来回答问题 . \n",
    suffix="Question: {input}\nAnswer:",
    input_variables=["input"],
)


chain = prompt | llm | output_parser

print(chain.invoke({"input": "李白和白居易谁活得更久"}))



Answer:  
Question: 李白和白居易谁活得更久?  
这里是否需要跟进问题：是的。  
追问：李白去世时几岁？  
中间答案：李白去世时61岁。  
追问：白居易去世时几岁？  
中间答案：白居易去世时74岁。  
所以最终答案是：白居易
